In [ ]:
pip install matplotlib seaborn pandas numpy

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#Estilo gráficas 
sns.set(style="darkgrid")
plt.rcParams['figure.figsize'] = (12, 6)

In [ ]:
# Carga de datos utilizando traducciones
train = pd.read_csv('sales_train.csv')
items = pd.read_csv('items_en.csv') 
cats = pd.read_csv('item_categories_en.csv')
shops = pd.read_csv('shops_en.csv')

print("filas y columnas train",train.shape)

In [ ]:
# Revisión de contenido 

print("Contenido TRAIN")
print("Columnas:", train.columns.tolist())
print(train.head(), "\n")

print("Contenido ITEMS")
print("Columnas:", items.columns.tolist())
print(items.head(), "\n")

print("Contenido CATS")
print("Columnas:", cats.columns.tolist())
print(cats.head(), "\n")

print("Contenido SHOP")
print("Columnas:", shops.columns.tolist())
print(shops.head(), "\n")

In [ ]:
# Uniones con tablas de traducciones
# train + items
train = pd.merge(train, items, on='item_id', how='left')

# train + categorías
train = pd.merge(train, cats, on='item_category_id', how='left')

# # train + shop
train = pd.merge(train, shops, on='shop_id', how='left')

# Formato fecha
train['date'] = pd.to_datetime(train['date'], format='%d.%m.%Y')


In [ ]:
# Verificación
print("filas y columnas train",train.shape)
train.head()

In [ ]:
# Estadísticas básicas 

#Numéricas
pd.set_option('display.float_format', lambda x: '%.2f' % x)
train.describe()



In [ ]:
#Categóricas
train.describe(include=['object'])

In [ ]:
# Datos faltantes

# Conteo nulos 
print("Valores nulos")
nulos = train.isnull().sum()
print(nulos[nulos > 0]) 



In [ ]:
# REVISIÓN INCIAL DE VENTAS

# Agrupar ventas por mes 
monthly_sales = train.groupby('date_block_num')['item_cnt_day'].sum()

# Gráficas 
plt.figure(figsize=(12, 5))
plt.plot(monthly_sales.index, monthly_sales.values, marker='o', color='b')
plt.title('Ventas ene 2013 - oct 2015')
plt.xlabel('Mes 0 = ene 2013')
plt.ylabel('Artículos vendidos')
plt.axvline(x=11, color='r', linestyle='--', label='dic 2013') 
plt.axvline(x=23, color='r', linestyle='--', label='dic 2014') 
plt.legend()
plt.show()



In [ ]:
# REVISIÓN OUTLIERS
# Para número de artículos vendidos y comportamiento de precios
fig, ax = plt.subplots(1, 2, figsize=(14, 4))

sns.boxplot(x=train['item_cnt_day'], ax=ax[0])
ax[0].set_title('Items vendidos por día')

sns.boxplot(x=train['item_price'], ax=ax[1])
ax[1].set_title('Precios')

plt.show()

In [ ]:
# Limpieza valores atípicos

# Eliminar precios negativos
train = train[train['item_price'] > 0]

# Eliminar precios muy altos
train = train[train['item_price'] < 100000]

#Eliminar número de unidades vendidas muy altas
train = train[train['item_cnt_day'] < 1000]



In [ ]:
# Devoluciones 
print(" % devoluciones:", (train[train['item_cnt_day'] < 0].shape[0] / train.shape[0]) * 100)

# Duplicados 

print("Duplicados:", train.duplicated().sum())
train = train.drop_duplicates()

print("Filas y columnas después de limpieza", train.shape)

In [ ]:
# Revisión de tiendas abiertas 
sales_by_shop = train.pivot_table(index='shop_id', columns='date_block_num', values='item_cnt_day', aggfunc='sum')

# Meses sin ventas 
sales_by_shop = sales_by_shop.fillna(0)

# Heat map
plt.figure(figsize=(20, 10))
sns.heatmap(sales_by_shop, cmap='viridis', vmin=0, vmax=2000) # vmin/vmax ajustan el contraste
plt.title('Ventas por tienda')
plt.xlabel('mes 0 = ene 2013')
plt.ylabel('ID Tienda')
plt.show()

In [ ]:
# Check 2 ventas por mes 

train['month'] = train['date'].dt.month

# Suma de ventas por mes
monthly_sales = train.groupby(['date_block_num', 'month'])['item_cnt_day'].sum().reset_index()

plt.figure(figsize=(12, 6))
sns.boxplot(x='month', y='item_cnt_day', data=monthly_sales)
plt.title('Distribución ventas por mes')
plt.xlabel('Mes (1=ene , 12= dic)')
plt.ylabel('Total de Ventas')
plt.show()

In [ ]:
# CATEGORÍAS TOP

# Ventas por categoría
cat_sales = train.groupby('item_category_name')['item_cnt_day'].sum().sort_values(ascending=False)

# Top 20
plt.figure(figsize=(12, 8))
sns.barplot(y=cat_sales.index[:20], x=cat_sales.values[:20], palette='magma')
plt.title('Top 20 categorías más vendidas')
plt.xlabel('# unidades vendidas')
plt.show()

In [ ]:
# Revisión de tiendas train vs test
test = pd.read_csv('test.csv')

# Tiendas 
tiendas_train = set(train['shop_id'].unique())
tiendas_test = set(test['shop_id'].unique())
print("Tiendas nuevas en test:", tiendas_test - tiendas_train)

# Productos 
items_train = set(train['item_id'].unique())
items_test = set(test['item_id'].unique())
print("Productos nuevos en test:", len(items_test - items_train))

In [ ]:
# CONSOLIDACIÓN DE INFORMACIÓN MES-> TIENDA -> PRODUCTO -> VENTAS

from itertools import product

# Matriz mes-tienda-item
matrix = []
cols = ['date_block_num','shop_id','item_id']

for i in range(34):
    sales = train[train.date_block_num==i]
    matrix.append(np.array(list(product([i], sales.shop_id.unique(), sales.item_id.unique())), dtype='int16'))

matrix = pd.DataFrame(np.vstack(matrix), columns=cols)
matrix['date_block_num'] = matrix['date_block_num'].astype(np.int8)
matrix['shop_id'] = matrix['shop_id'].astype(np.int8)
matrix['item_id'] = matrix['item_id'].astype(np.int16)
matrix.sort_values(cols, inplace=True)

# Incluimos las ventas por mes
group = train.groupby(['date_block_num','shop_id','item_id']).agg({'item_cnt_day': ['sum']})
group.columns = ['item_cnt_month']
group.reset_index(inplace=True)

matrix = pd.merge(matrix, group, on=cols, how='left')

# Reemplazamos nulos por 0
matrix['item_cnt_month'] = (matrix['item_cnt_month']
                                .fillna(0)
                                .clip(0,20) # Limitamos a 20 como pide la competencia
                                .astype(np.float16))

print("Dimensiones matriz:", matrix.shape)
matrix.head()

In [ ]:
# Filas con 0 ventas
ceros = matrix[matrix['item_cnt_month'] == 0].shape[0]
total = matrix.shape[0]
porcentaje_sin_ventas = (ceros / total) * 100

print(f"Total de combinaciones Mes-Tienda-Producto: {total}")
print(f"Combinaciones con 0 ventas: {ceros}")
print(f"Porcentaje sin ventas: {porcentaje_sin_ventas:.2f}%")